# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

d:\anaconda3\envs\chatglm3-demo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda3\envs\chatglm3-demo\lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- modeling_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


bin d:\anaconda3\envs\chatglm3-demo\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.41it/s]


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


## 使用微调前 ChatGLM3

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、干燥、强健、积极向上等含义。乾卦的卦辞是“元、亨、利、永”，表示这个卦具有正统、通晓、吉祥、长久等特点。

乾卦的六爻分别代表着不同的阶段和境遇，从初爻到六爻，依次是：初爻为“复”，表示万事万物的起始，二爻为“临”，意味着面对挑战和困难时要有勇气和决心，三爻为“开水”，象征着事物发展迅速，四爻为“复”，意味着回到原来的状态，五爻为“观”，表示从长远来看事物的发展趋势，六爻为“永”，意味着永远保持现状。

乾卦在五行中代表着金，因此在中医、道家、儒家等传统文化中具有重要的地位。同时，乾卦在现代社会中也具有启示作用，鼓励人们勇往直前，积极向上，克服困难，实现目标。


In [8]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是《易经》中第64卦，由两个阴爻夹一个阳爻构成，象征着地、水、军队等含义。师卦的卦辞是“元、亨、利、贞”，表示这个卦具有正统、通晓、吉祥、正统等特点。

师卦的六爻分别代表着不同的阶段和境遇，从初爻到六爻，依次是：初爻为“复”，表示万事万物的起始，二爻为“临”，意味着面对挑战和困难时要有勇气和决心，三爻为“开水”，象征着事物发展迅速，四爻为“复”，意味着回到原来的状态，五爻为“观”，表示从长远来看事物的发展趋势，六爻为“永”，意味着永远保持现状。

师卦在五行中代表着水，因此在中医、道家、儒家等传统文化中具有重要的地位。同时，师卦在现代社会中也具有启示作用，鼓励人们勇往直前，积极向上，克服困难，实现目标。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [9]:
from peft import PeftModel, PeftConfig

epochs = 50
timestamp = '20240206_210859'
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [10]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [11]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的一种卦象，卦象由六个阳爻（即：阳刚之爻）在上，一个阴爻在下的排列构成。这种排列方式被称为“乾”，即“天”。

乾卦的含义非常丰富，它象征着宇宙、生活、权力、 strength、刚毅等。乾卦表示宇宙中万物生长、发展、壮大的过程，也象征着人们应该具有刚毅、决断、勇敢、 strong 的品质。

在《易经》中，乾卦的卦象是由两个阴爻夹一个阳爻构成，这种阴爻夹阳的排列方式，象征着阴阳相济，刚柔并济。因此，乾卦也象征着人们应该保持内心的阴柔，同时保持坚定的信念和决断力。

乾卦的卦象是由三个阳爻夹一个阴爻构成，这种阳多阴少的排列方式，象征着阳性的力量和刚毅。因此，乾卦也象征着人们应该具有坚定的信念和决断力，同时保持内心的柔软和善良。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是《易经》中的第一卦，也是六十四卦中的首卦。乾卦是由两个阳爻夹一个阴爻构成，象征着天，反映了宇宙的运行和自然的法则。乾卦的卦象为：上阴下阳，阳爻在顶，阴爻在底。乾卦的卦辞为：“元，亨，利，贞”，表示乾卦具有万物发展的愿望，可以获得成功，是一种吉卦。



In [12]:
base_response, ft_response = compare_chatglm_results(query="地水师卦")

问题：地水师卦

原始输出：
《易经》中的“师卦”是卦象为师（师：众，众疾之疾），其象征者多，其吉凶者悖。师卦•象辞：师，众疾之疾，先吉后凶，舍去旧，取舍新。 bears the qualities of the earth, and its influence is beneficial.师卦•断卦：初爻为师，险，次之吉，舍旧谋新。 上吉，无咎，勿忧。北宋易学家邵雍卦象：万物劳苦，师出相生，舍旧谋新，先凶后吉。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [13]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》hexagram（卦象）之一，卦象上下一阳一阴，阴爻在阳爻之上，阴柔不能胜阳，于是争竞诉讼，体现了一种不能和谐相处的状态。讼卦象征诉讼争端，同时也提醒人们要合理处理矛盾和纠纷，避免争端。

讼卦的卦象是：下卦为阴，上卦为阳。

卦象解释：讼卦卦象上下一阳一阴，阴爻在阳爻之上，表示虽然阴爻处于优势地位，但由于阳刚的力量，阴不能胜阳，于是出现争竞诉讼的情况。

卦象意义：讼卦提醒人们要合理处理矛盾和纠纷，避免争端。它告诉我们要以和为贵，以和平共处为原则，才能保持和谐顺利。

基本卦象：阴（下）+阳（上）=阴胜阳负，表示争端，诉讼，矛盾。


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [13]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240206_211504"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [14]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B (Epoch=3, automade-dataset) 微调后：\n{ft_response}")
    return base_response, ft_response

In [15]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它反映了宇宙中充满活力和力量的因素，以及积极向上、充满生命力的态度。在卜卦中，乾卦预示着顺利，提示人们要坚定刚健，积极进取，克服困难。

在儒家经典《周易》中，乾卦的卦辞描述了天的情况，指出天是刚健强劲的，然而也有柔的一面。这一卦的取义在于强调阳刚之美，提示人们要积极向前，勇攀高峰。

在易象中，乾卦代表天，表示宇宙中充满活力和力量的因素。它反映了宇宙中积极向上、充满生命力的态度。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，包含着万物运行的刚健与强劲，象征着天地的强大与永恒。在周易中，乾卦代表天，代表着刚强、强劲和强大，象征着宇宙的运行和万物的生生不息。同时，乾卦也代表着正义、公正、坚定和果敢，具有强烈的个人主义色彩。

在周易中，乾卦的卦辞描述了天地的强大与永恒，强调刚强与强劲，象征着宇宙的运行和万物的生生不息。同时，乾卦也代表着正义、公正、坚定和果敢，具有强烈的个人主义色彩。

在易学中，乾卦代表着刚强、强劲和强大的象征，强调坚贞不屈解，解卦中提到健解，即困难中能够解救。同时，乾卦也代表着正义、公正、坚定和果敢，具有强烈的个人主义色彩。在解卦中，需要强调的是，乾卦所象征的刚强、强劲和强大，并不是指人力可以实现的效果，而是需要强调坚贞不屈解，解卦中提到困难中能够解救。

总结起来，乾卦是周易中的一卦，代表天，包含着万物运行的刚健与强劲，象征着天地的强大与永恒。在周易中，乾卦代表天，代表着刚强、强劲和强大，象征着宇宙的运行和万物的生生不息。同时，乾卦也代表着正义、公正、坚定和果敢，具有强烈的个人主义色彩。


In [17]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，由两个卦象组合而成：上卦为坤，代表大地，代表顺从、顺应天道的人；下卦为坎，代表水，代表变化、灵活。

地水师卦的核心卦象是“地水相成”，代表水在地下流动，大地吸收水分，象征着顺利融合双方的力量，代表着顺利和谐地处理双方的矛盾。

在占卜时，地水师卦预示着会遇到困难，必须 flexibility（灵活），求变，代表变通，求变则通。代表预测：困难已至，宜速变通，宜谨慎行事。

地水师卦的核心哲学是：调料需调和使用， wealth（财富）需谨慎使用，二者必需相互配合。

地水师卦适用于占卜、预测未来的情况，提醒人们遇到困难和变故，必须谨慎处理，变通求通。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）和坤卦（地）组成的卦象，代表地下的水，象征润泽大地的力量。根据《易经》中的解释，地水师卦预示着吉祥如意，具有强大的力量，可以克服各种困难。

根据传统解卦，地水师卦分为两个卦象，第一个卦象是坎卦（水），代表灵活变化的力量，具有吉祥如意，靠智慧取悦于人；第二个卦象是坤卦（地），代表刚健柔顺，具有强大的力量，可以克服各种困难。

地水师卦的核心理念是：吉祥如意，靠智慧取悦于人，靠刚健柔顺的力量，可以克服各种困难。


In [17]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
讼卦是周易卦象中的一枚卦，由上卦坎（水）和下卦乾（天）组成，预示着天卦和地卦的矛盾。根据《象辞》，讼卦表示天下一卦，涉及利害的变化，以及充满着有所行动的象。

根据《象辞》，讼卦的初始装饰是：上坎下乾。上坎代表水，象征柔弱之形，表示盒中蓄水，其味甜美，其性温良。下乾代表天，象征刚健之形，其形高峻，其性刚硬。 initial

讼卦的卦象组合意味着利害关系，双方都期望得到自己的利益，但互相对立，难以相让。因此，讼卦预示着双方互相攻击，难以取得胜利。在《象辞》中，讼卦被解释为：双方互相对立，难以相让，象征纠纷难以解决。

讼卦的运势和运势 linked to the卦象 and the circumstances surrounding it. According to the ancient I Ching, the讼卦 was predicted to indicate a dispute between parties, and the outcome would depend on the actions and decisions made by both parties. The ancient I Ching also predicted that, in the end, both parties would be willing to submit to the other, and the dispute would be resolved.

讼卦的哲学原理源于《易经》中的 "的天行健，君子以自强不息；地势坤，君子以厚德载物.", 

讼卦的解析需要结合《象辞》和《象形》来综合解读，解卦需要结合当事人的处境和动作，以及对手的处境和动作来进行。

讼卦的结局取决于双方是否愿意相互谦让，以达到和谐共赢。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是周易卦象中的一枚卦，由下卦坤（地）上卦坎（水）组成，预示着利害相间的景象。在卜卦时，讼卦表示当前情况并不顺利，可能会遇到一些困难或问题。

讼卦的核心哲学是：天行健，君子以自强不息；君子鸣琴而治国家，修身养性者，可以国家大事。

讼卦的核心指数为：

- 事业：讼卦

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [24]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240206_212449"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [19]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [20]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由六个阳爻组成，象征着天。它代表着刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象,从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [27]:
base_response, ft_response = compare_chatglm_results(query="师卦是什么？")

问题：师卦是什么？

原始输出：
{'name': '师卦是由坎卦（水）和坤卦（地）相叠而成。在这个预测过程中，地中有水，预示着君子观此卦象，应当柔顺地对待事物。', 'content': '\n师卦的核心哲学是：柔胜刚，高度地重视内在素质，潜在地发挥自己的长处，待机而动。在事业上，字母“师”寓指军队，说明 postal code 2。在商业领域，寓言中描绘了君子出使外国，得胜而归，预示着 local government 1。在决策中，象征着智慧与权谋的结合。\n\n师卦的吉凶取决于观者的心地和行动。如果观者心怀正念、行为合乎道德准则，那么卦象吉祥。相反，若观者心志不诚、行为不检，则易招祸端。\n\n师卦的卦象既包含了水的柔美，也包含了地的刚毅，寓意着 flexibility and resilience。因此，在事业和商业中， depending on the situation, both flexibility and resilience are needed. In personal relationships, the卦 suggests that love and understanding are the keys to a happy union.'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


In [30]:
base_response, ft_response = compare_chatglm_results(query="讼卦是什么？")

问题：讼卦是什么？

原始输出：
{'name': '挂一爻，以观其意，则梦见一 commit in the night, and he knew it was going to be a big commitment. A positive result from the诉讼卦。', 'content': '\n ESG and sustainability have become increasingly important in the investment world, and the Chinese government has been actively promoting the development of a green and sustainable economy. As a result, the legal industry is facing increasing pressure to adapt to these changing circumstances, and there is a growing demand for legal professionals with expertise in ESG and sustainability.\n\nThe commit in the dream may represent a legal commitment or promise, which can be both positive or negative depending on the context. However, regardless of its nature, the dream suggests that you are taking a big step forward, and the outcome will be favorable if you approach it with confidence and careful planning.\n\nOverall, the commit in the dream is a reminder that the legal industry must adapt to the changing times and embrace ESG and su